<a href="https://colab.research.google.com/github/Daniel-MDA/Data-Analysis-with-Python-Pandas/blob/main/Regressao_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab04_Regressao_Linear_01

### Disciplina Inteligência Artificial

### Prof: José Humberto Cruvinel

### Nome dos alunos:

* Daniel Moacir de Ávila

Neste jupyter vamos criar um modelo de regressão linear para estimar a probabilidade de admissões de pós-graduação.

Para correto funcionamento, crie no seu drive a pasta **arquivos_colab/Lab04** e copie para esta pasta o arquivo Admission_Predict_Ver1.1.csv que está em https://drive.google.com/drive/folders/10Lgd1c5q8SG0zN5nHG-XPQauwUHJOUW6?usp=sharing

Dataset: https://www.kaggle.com/mohansacharya/graduate-admissions

O conjunto de dados contém vários parâmetros que são considerados importantes durante a aplicação de programas de mestrado. Os parâmetros incluídos são:

* Pontuações GRE (de 340)
* Pontuações do TOEFL (de 120)
* Classificação da universidade (em 5)
* Declaração de finalidade e força da carta de recomendação (em 5)
* Graduação GPA (de 10)
* Experiência de pesquisa (0 ou 1)
* Chance de admissão (variando de 0 a 1) - TARGET

Este jupyter notebook deve ser executado no Google Colab (https://colab.research.google.com/) 

## Montando o Drive

In [ ]:
# Execute esta célula para montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/arquivos_colab/Lab04

In [ ]:
%ls -la

## Carregando o dataset

In [ ]:
# Carregando Bibliotecas Python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline 

In [ ]:
# Carregando o Dataset Admission_Predict_Ver1.1.csv com o pacote Pandas, criando o DataFrame df
df = pd.read_csv("Admission_Predict_Ver1.1.csv")
df.head()

In [ ]:
# Verificando os últimos registros do dataset
df.tail()

Veja acima que a primeira coluna (Serial No.) podemos ignorar, pois é apenas uma chave e que não afeta a chance de admissão

In [ ]:
# Ignorando a primeia linha
df = df.iloc[:,1:]
df.head()

Veja acima que a primeira coluna foi removida

In [ ]:
# Vamos verificar se existem valores nulos
df.isna().sum()

Veja acima que não temos nenhum valor nulo

In [ ]:
# Imprimindo o describe()
df.describe()

Como pode ser visto acima, temos 500 exemplos

In [ ]:
# Vamos verificar o nome das colunas
df.columns

In [ ]:
# Vamos renomear as colunas para remover espaços
df.columns = ['GRE_Score', 'TOEFL_Score', 'University_Rating', 'SOP', 'LOR', 'CGPA', 'Research', 'Chance_of_Admit']
df.head()

In [ ]:
# Vamos ver o Shape dos dados
print('Shape:',df.shape)

Temos aqui 500 linhas com 8 colunas, entretanto a última coluna é a coluna TARGET (ALVO)

## Explorando o Dataset

In [ ]:
# Vamos explorar os dados de resultado
# Calculando a média da variável target
media_esperada = df['Chance_of_Admit'].mean()
media_esperada

In [ ]:
# Histograma do target
df.hist(column='Chance_of_Admit')

In [ ]:
# Vamos plotar o DataFrame para ver se encontramos alguma correlação entre as variáveis que determinam o resultado
df.plot()

Não foi possível encontrar uma relação. Vamos tentar de outra forma

In [ ]:
# Criando uma função para plotar a correlação entre as variáveis
def visualize_correlation_matrix(data, hurdle = 0.0):
    R = np.corrcoef(data, rowvar = 0)
    R[np.where(np.abs(R) < hurdle)] = 0.0
    heatmap = plt.pcolor(R, cmap = mpl.cm.coolwarm, alpha = 0.8)
    heatmap.axes.set_frame_on(False)
    heatmap.axes.set_yticks(np.arange(R.shape[0]) + 0.5, minor = False)
    heatmap.axes.set_xticks(np.arange(R.shape[1]) + 0.5, minor = False)
    heatmap.axes.set_xticklabels(variables, minor = False)
    plt.xticks(rotation=90)
    heatmap.axes.set_yticklabels(variables, minor = False)
    plt.tick_params(axis = 'both', which = 'both', bottom = False, top = False, left = False, right = False) 
    plt.colorbar()
    plt.show()

In [ ]:
# Vamos gerar a matriz de correlação entre as variáveis
matriz_corr = df.corr()
observations = len(df)
variables = df.columns
visualize_correlation_matrix(df, hurdle = 0.5)

Veja que as variáveis CGPA, GRE_Score e TOEFL_Score tem maior influência em Chance_of_Admit

In [ ]:
# Vamos separar as variáveis (X) do resultado (y)
X = df.iloc[:,:-1]
y = df['Chance_of_Admit'].values

In [ ]:
# Imprimindo os primeiros valores de X
X.head()

In [ ]:
# Imprimindo os 10 primeiros valores de y
print(y[:10])

In [ ]:
# Imprimindo o Shape de X e y
print(X.shape)
print(y.shape)

In [ ]:
# Vamos agora calcular a covariância entre as variáveis
import statsmodels.regression.linear_model as sm
modelo_v1 = sm.OLS(y, X)
modelo_v2 = modelo_v1.fit()
modelo_v2.summary()

Explicação dos dados acima:

* std err: O erro padrão básico da estimativa do coeficiente. 
* t: O valor da estatística t. Esta é uma medida de quão estatisticamente significativo é o coeficiente.
* P> | t |: P-valor de que a hipótese nula de que o coeficiente = 0 é verdadeira. Se for menor que o nível de confiança, geralmente 0,05, indica que há uma relação estatisticamente significativa entre o termo e a resposta.

Podemos ver que CGPA é muito significativo

## Regressão Linear com 1 variável

In [ ]:
# Vamos agora imprimir um scatter plot entre CGPA e o resultado (target)
# Definindo o range dos valores de x e y
VAR = 'CGPA'
TARGET = 'Chance_of_Admit'
x_range = [df[VAR].min(), df[VAR].max()]
y_range = [df[TARGET].min(), df[TARGET].max()]
# Plot dos valores de x e y com a média
scatter_plot = df.plot(kind = 'scatter', x = VAR, y = TARGET, xlim = x_range, ylim = y_range)

In [ ]:
X[VAR].values.shape

In [ ]:
# Separando apenas a variável de interesse e fazendo o reshape
X_1 = X[VAR].values.reshape((len(df), 1))
print(X_1.shape)

In [ ]:
# Separação entre os dados de treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (X_1 , y, test_size=0.2, random_state=50)
print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_test:',X_test.shape)
print('y_test:',y_test.shape)

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
# Agora efetivamente treinamos o modelo com fit()
lin_reg.fit(X_train, y_train)

In [ ]:
# prevendo valores para X_test
y_pred = lin_reg.predict(X_test)
print('Shape y_pred:',y_pred.shape)

In [ ]:
# Medidas de desempenho
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
print("Mean Squared Error (MSE):",mean_squared_error(y_pred,y_test))
print("Mean Absolute Error (MAE):",mean_absolute_error(y_pred,y_test))
print("Score R2:",r2_score(y_pred,y_test))
print('Score:',lin_reg.score(X_test, y_test))

In [ ]:
# Plotando a linha de regressão para todos os dados

# Prevendo os valores de y para todos os dados
y_pred_all = lin_reg.predict(X_1)

# Primeira camada do Scatter Plot
scatter_plot = df.plot(kind = 'scatter', x = VAR, y = TARGET, xlim = x_range, ylim = y_range)

# Segunda camada do Scatter Plot (linha de regressão)
regression_line = scatter_plot.plot(df[VAR], y_pred_all, 'o', color = 'orange', linewidth = 2)

## Regressão Linear com mais variáveis

In [ ]:
# Vamos utilizar 3 variáveis
VAR = ['CGPA', 'GRE_Score', 'TOEFL_Score']
X_2 = X[VAR].values.reshape((len(df), 3))
print(X_2.shape)

In [ ]:
# Separação entre os dados de treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (X_2 , y, test_size=0.2, random_state=50)
print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_test:',X_test.shape)
print('y_test:',y_test.shape)

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
# prevendo valores para X_test
y_pred = lin_reg.predict(X_test)
print('Shape y_pred:',y_pred.shape)

In [ ]:
# Medidas de desempenho
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
print("Mean Squared Error (MSE):",mean_squared_error(y_pred,y_test))
print("Mean Absolute Error (MAE):",mean_absolute_error(y_pred,y_test))
print("Score R2:",r2_score(y_pred,y_test))
print('Score:',lin_reg.score(X_test, y_test))

Veja que o uso de mais variáveis melhorou o modelo, pois o valor de R2 aumentou e os erros médios diminuíram. 

# Exercício (Lab04)

Você viu acima que o uso de mais variáveis na regressão linear melhorou o modelo.

Adicione novas células de código abaixo e implemente um modelo de Regressão Linear que utilize todas as variáveis do dataset para prever a variável TARGET.

Crie também uma célula de código para imprimir os valores de MSE, MAE e R2

Finalmente, crie uma célula de Texto para responder se o resultado foi melhor ou pior.

Obs: Antes de entregar seu notebook, verifique se o mesmo está funcionando adequadamente com o menu "Ambiente de Execução > Reiniciar e executar tudo".

Salve o arquivo .ipynb com a opção "Arquivo > Fazer download do .ipynb" e poste pelo Ulife

FIM